# Crypto-Trust: Bitcoin OTC Network Analysis

## Business Problem
**Objective**: Detect fraud rings and identify high-trust anchors in a Bitcoin OTC trading network.

**Context**: Bitcoin OTC is a peer-to-peer trading platform where users rate each other to build reputation. We need to distinguish between:
- **Trust Anchors**: Genuine power users who are reliable trading partners
- **Fraud Clusters**: Potential Sybil attack rings (fake accounts rating each other)

**Data**: Signed directed network with ratings from -10 (distrust) to +10 (trust)

---

## 1. Setup and Imports

In [ ]:
# Core Libraries
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Community Detection
import community.community_louvain as community_louvain

# Visualization Settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ All libraries loaded successfully")
print(f"NetworkX version: {nx.__version__}")
print(f"Pandas version: {pd.__version__}")

## 2. Data Loading and Preprocessing

**Business Logic**: Load the raw trust network data and prepare it for graph analysis.

**Schema**:
- `source`: User who gave the rating
- `target`: User who received the rating
- `rating`: Trust score (-10 to +10)
- `time`: Unix timestamp

In [ ]:
# Load the dataset
df = pd.read_csv('soc-sign-bitcoinotc.csv', 
                 names=['source', 'target', 'rating', 'time'])

print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nFirst 5 rows:")
df.head()

In [ ]:
# Data Quality Check
print("=== Data Quality Report ===")
print(f"Total edges: {len(df):,}")
print(f"Unique users (source): {df['source'].nunique():,}")
print(f"Unique users (target): {df['target'].nunique():,}")
print(f"Unique users (combined): {len(set(df['source']) | set(df['target'])):,}")
print(f"\nRating range: [{df['rating'].min()}, {df['rating'].max()}]")
print(f"Missing values: {df.isnull().sum().sum()}")
print(f"\nRating distribution:")
print(df['rating'].value_counts().sort_index())

## 3. Graph Construction

**Critical**: Many graph algorithms (PageRank, Dijkstra) don't support negative weights.

**Solution**: Create three graph views:
- `G`: Full network (all edges)
- `G_trust`: Only positive ratings (for PageRank, shortest path)
- `G_distrust`: Only negative ratings (for fraud analysis)

In [ ]:
# Create the main directed graph
G = nx.DiGraph()

# Add edges with attributes
for _, row in df.iterrows():
    G.add_edge(row['source'], row['target'], 
               rating=row['rating'], 
               time=row['time'],
               weight=abs(row['rating']))  # Weight for algorithms

print(f"✓ Main graph G created: {G.number_of_nodes():,} nodes, {G.number_of_edges():,} edges")

In [ ]:
# Create Trust Subgraph (positive ratings only)
trust_edges = [(u, v, d) for u, v, d in G.edges(data=True) if d['rating'] > 0]
G_trust = G.edge_subgraph(trust_edges).copy()

print(f"✓ Trust graph G_trust created: {G_trust.number_of_nodes():,} nodes, {G_trust.number_of_edges():,} edges")

# Create Distrust Subgraph (negative ratings only)
distrust_edges = [(u, v, d) for u, v, d in G.edges(data=True) if d['rating'] < 0]
G_distrust = G.edge_subgraph(distrust_edges).copy()

print(f"✓ Distrust graph G_distrust created: {G_distrust.number_of_nodes():,} nodes, {G_distrust.number_of_edges():,} edges")

## 4. Descriptive Analytics

**Business Insight**: Understanding the network structure helps identify patterns of trust and distrust.

In [ ]:
# Network Statistics Summary
print("=" * 60)
print("NETWORK STATISTICS SUMMARY")
print("=" * 60)

stats = pd.DataFrame({
    'Graph': ['Full Network (G)', 'Trust Network (G_trust)', 'Distrust Network (G_distrust)'],
    'Nodes': [G.number_of_nodes(), G_trust.number_of_nodes(), G_distrust.number_of_nodes()],
    'Edges': [G.number_of_edges(), G_trust.number_of_edges(), G_distrust.number_of_edges()],
    'Density': [nx.density(G), nx.density(G_trust), nx.density(G_distrust)]
})

print(stats.to_string(index=False))
print("\nKey Insight: Trust edges dominate the network, showing healthy platform activity.")

In [ ]:
# Rating Distribution - The "J-Curve" of Trust
fig, ax = plt.subplots(figsize=(12, 6))

rating_counts = df['rating'].value_counts().sort_index()
colors = ['red' if r < 0 else 'lightcoral' if r == 0 else 'green' for r in rating_counts.index]

ax.bar(rating_counts.index, rating_counts.values, color=colors, edgecolor='black', alpha=0.7)
ax.set_xlabel('Rating Score', fontsize=12, fontweight='bold')
ax.set_ylabel('Frequency (log scale)', fontsize=12, fontweight='bold')
ax.set_title('Rating Distribution: The J-Curve of Trust', fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.grid(axis='y', alpha=0.3)
ax.axvline(x=0, color='black', linestyle='--', linewidth=2, label='Neutral')
ax.legend()

plt.tight_layout()
plt.show()

print("\n📊 Business Insight:")
print("The J-curve shows that users overwhelmingly give high trust ratings (+10),")
print("with very few distrust ratings. This is typical of reputation systems.")

### Top Users by Activity

In [ ]:
# Most Active Raters (Out-Degree)
out_degrees = dict(G.out_degree())
top_raters = sorted(out_degrees.items(), key=lambda x: x[1], reverse=True)[:5]

print("=" * 60)
print("TOP 5 MOST ACTIVE RATERS (Out-Degree)")
print("=" * 60)
for rank, (user, degree) in enumerate(top_raters, 1):
    print(f"{rank}. User {user}: {degree} ratings given")

print("\n💡 These users are highly engaged in rating others.")

In [ ]:
# Most Rated Users (In-Degree)
in_degrees = dict(G.in_degree())
top_rated = sorted(in_degrees.items(), key=lambda x: x[1], reverse=True)[:5]

print("=" * 60)
print("TOP 5 MOST RATED USERS (In-Degree)")
print("=" * 60)
for rank, (user, degree) in enumerate(top_rated, 1):
    print(f"{rank}. User {user}: {degree} ratings received")

print("\n💡 These users are well-known and frequently rated by the community.")

## 5. Algorithm 1: Trust Anchor Identification (PageRank)

**Business Logic**: PageRank identifies users who receive trust from other high-trust users.

**Why PageRank > In-Degree?**
- In-Degree counts raw number of ratings (vulnerable to spam)
- PageRank weights ratings by the trustworthiness of the rater
- Result: Filters out low-quality spam ratings

**Technical Note**: Run on `G_trust` only (PageRank requires positive weights)

In [ ]:
# Calculate PageRank on Trust Network
pagerank_scores = nx.pagerank(G_trust, weight='weight')

# Get top 10 by PageRank
top_pagerank = sorted(pagerank_scores.items(), key=lambda x: x[1], reverse=True)[:10]

print("=" * 60)
print("TOP 10 TRUST ANCHORS (PageRank)")
print("=" * 60)
for rank, (user, score) in enumerate(top_pagerank, 1):
    in_deg = G_trust.in_degree(user)
    print(f"{rank}. User {user}: PageRank={score:.6f}, In-Degree={in_deg}")

print("\n✓ These are the most trustworthy users based on network influence.")

In [ ]:
# Comparison: PageRank vs In-Degree
print("\n" + "=" * 60)
print("BUSINESS INSIGHT: PageRank vs. In-Degree")
print("=" * 60)

top5_pagerank = [user for user, _ in top_pagerank[:5]]
top5_indegree = [user for user, _ in top_rated[:5]]

comparison_df = pd.DataFrame({
    'Rank': range(1, 6),
    'PageRank Top Users': top5_pagerank,
    'PageRank Score': [f"{pagerank_scores[u]:.6f}" for u in top5_pagerank],
    'In-Degree Top Users': top5_indegree,
    'In-Degree Count': [in_degrees[u] for u in top5_indegree]
})

print(comparison_df.to_string(index=False))

overlap = set(top5_pagerank) & set(top5_indegree)
print(f"\n📊 Overlap: {len(overlap)} users appear in both Top 5 lists")
print("\n💡 Key Insight:")
print("PageRank identifies users trusted by other trustworthy users (quality),")
print("while In-Degree only counts total ratings (quantity).")
print("Users appearing in PageRank but not In-Degree are 'hidden gems' - ")
print("trusted by key influencers despite fewer total ratings.")

## 6. Algorithm 2: Fraud Ring Detection (Community Detection)

**Business Logic**: Sybil attacks create clusters of fake accounts that rate each other.

**Detection Strategy**:
1. Use Louvain algorithm to detect communities
2. Identify **Suspicious Isolated Communities**:
   - Small size (< 20 users)
   - Dense internal connections
   - Zero connections to the main network

**Technical Note**: Louvain requires undirected graph

In [ ]:
# Convert to undirected for Louvain
G_trust_undirected = G_trust.to_undirected()

# Run Louvain Community Detection
partition = community_louvain.best_partition(G_trust_undirected, weight='weight')

# Analyze communities
community_sizes = Counter(partition.values())

print("=" * 60)
print("COMMUNITY DETECTION RESULTS")
print("=" * 60)
print(f"Total communities detected: {len(community_sizes)}")
print(f"\nCommunity size distribution:")
print(f"  - Communities with 1 user: {sum(1 for s in community_sizes.values() if s == 1)}")
print(f"  - Communities with 2-10 users: {sum(1 for s in community_sizes.values() if 2 <= s <= 10)}")
print(f"  - Communities with 11-50 users: {sum(1 for s in community_sizes.values() if 11 <= s <= 50)}")
print(f"  - Communities with 50+ users: {sum(1 for s in community_sizes.values() if s > 50)}")

# Find largest community
largest_community = max(community_sizes.items(), key=lambda x: x[1])
print(f"\nLargest community: ID {largest_community[0]} with {largest_community[1]} users")

In [ ]:
# Find the largest connected component (main network)
largest_cc = max(nx.connected_components(G_trust_undirected), key=len)

# Identify Suspicious Isolated Communities
suspicious_communities = []

for comm_id, size in community_sizes.items():
    if size < 20:  # Small communities
        # Get users in this community
        comm_users = [user for user, c in partition.items() if c == comm_id]
        
        # Check if isolated from main component
        connections_to_main = 0
        for user in comm_users:
            neighbors = set(G_trust_undirected.neighbors(user))
            if neighbors & largest_cc:
                connections_to_main += 1
                break
        
        if connections_to_main == 0:
            suspicious_communities.append((comm_id, size, comm_users))

print("\n" + "=" * 60)
print("🚨 POTENTIAL FRAUD RINGS DETECTED")
print("=" * 60)
print(f"Suspicious isolated communities found: {len(suspicious_communities)}")

if suspicious_communities:
    print("\nTop 10 suspicious communities:")
    for i, (comm_id, size, users) in enumerate(sorted(suspicious_communities, 
                                                       key=lambda x: x[1], 
                                                       reverse=True)[:10], 1):
        print(f"{i}. Community {comm_id}: {size} users (Users: {users[:5]}...)")
    
    print("\n💡 Business Recommendation:")
    print("These small, isolated communities may be Sybil attack rings.")
    print("Recommend manual review for users in these communities.")
else:
    print("\n✅ No suspicious isolated communities detected.")
    print("The network appears healthy with good connectivity.")

## 7. Algorithm 3: Trust Path Analysis (Shortest Path)

**Business Logic**: Can a new/random user establish a trust path to a gold-standard anchor?

**Risk Assessment**:
- **Path exists**: User is connected to trusted network → Lower risk
- **No path**: User is isolated → Higher risk

**Use Case**: Pre-trade risk scoring for new users

In [ ]:
# Select the Gold Standard User (highest PageRank)
gold_standard = top_pagerank[0][0]
print(f"🏆 Gold Standard User: {gold_standard} (PageRank: {pagerank_scores[gold_standard]:.6f})")

# Select a random low-degree user
low_degree_users = [u for u, d in G_trust.in_degree() if 1 <= d <= 3]
if low_degree_users:
    random_user = np.random.choice(low_degree_users)
else:
    # Fallback to any user if no low-degree users exist
    random_user = np.random.choice(list(G_trust.nodes()))

print(f"🎲 Random Low-Degree User: {random_user} (In-Degree: {G_trust.in_degree(random_user)})")

In [ ]:
# Compute Shortest Trust Path
print("\n" + "=" * 60)
print("TRUST PATH ANALYSIS")
print("=" * 60)

try:
    # Find shortest path
    path = nx.shortest_path(G_trust, source=random_user, target=gold_standard)
    path_length = len(path) - 1
    
    print(f"✓ Trust path found!")
    print(f"  Path length: {path_length} hops")
    print(f"  Path: {' → '.join(map(str, path))}")
    
    # Calculate total trust score along path
    total_trust = sum(G_trust[path[i]][path[i+1]]['rating'] for i in range(len(path)-1))
    avg_trust = total_trust / path_length
    
    print(f"\n  Total trust score along path: {total_trust}")
    print(f"  Average trust per hop: {avg_trust:.2f}")
    
    # Risk Assessment
    print("\n📊 Business Insight - Risk Assessment:")
    if path_length <= 2:
        print("  🟢 LOW RISK: User is closely connected to gold standard")
    elif path_length <= 4:
        print("  🟡 MEDIUM RISK: User is moderately connected")
    else:
        print("  🟠 ELEVATED RISK: User is distant from trusted network")
    
    if avg_trust >= 8:
        print("  ✓ Path has strong trust ratings")
    elif avg_trust >= 5:
        print("  ⚠ Path has moderate trust ratings")
    else:
        print("  ⚠ Path has weak trust ratings")

except nx.NetworkXNoPath:
    print(f"❌ No trust path exists from User {random_user} to Gold Standard {gold_standard}")
    print("\n📊 Business Insight:")
    print("  🔴 HIGH RISK: User is completely isolated from the trusted network")
    print("  Recommendation: Require additional verification before allowing trades")
    print("  This user may be:")
    print("    - A new user who hasn't built reputation yet")
    print("    - Part of an isolated community (potential fraud ring)")
    print("    - A dormant account")

except nx.NodeNotFound as e:
    print(f"❌ Error: Node not found in trust graph - {e}")
    print("This user may only have distrust ratings.")

## 8. Gephi Export

**Objective**: Export network with rich attributes for advanced visualization in Gephi.

**Node Attributes**:
- `pagerank`: Trust score (for node sizing)
- `community`: Community ID (for node coloring)
- `in_degree`: Number of ratings received
- `out_degree`: Number of ratings given

**Edge Attributes**:
- `rating`: Original rating score
- `rating_class`: Positive/Negative/Neutral (for edge coloring)
- `weight`: Absolute rating value

In [ ]:
# Prepare node attributes
for node in G.nodes():
    # PageRank (default 0 if not in trust network)
    G.nodes[node]['pagerank'] = pagerank_scores.get(node, 0.0)
    
    # Community (default -1 if not in trust network)
    G.nodes[node]['community'] = partition.get(node, -1)
    
    # Degree metrics
    G.nodes[node]['in_degree'] = G.in_degree(node)
    G.nodes[node]['out_degree'] = G.out_degree(node)

print("✓ Node attributes added: pagerank, community, in_degree, out_degree")

In [ ]:
# Prepare edge attributes
for u, v, data in G.edges(data=True):
    rating = data['rating']
    
    # Classify rating
    if rating > 0:
        data['rating_class'] = 'Positive'
    elif rating < 0:
        data['rating_class'] = 'Negative'
    else:
        data['rating_class'] = 'Neutral'

print("✓ Edge attributes added: rating_class")

In [ ]:
# Export to GEXF format
output_file = 'bitcoin_trust_network.gexf'
nx.write_gexf(G, output_file)

print(f"\n✅ Network exported to: {output_file}")
print(f"\nFile stats:")
print(f"  - Nodes: {G.number_of_nodes():,}")
print(f"  - Edges: {G.number_of_edges():,}")
print(f"  - Node attributes: pagerank, community, in_degree, out_degree")
print(f"  - Edge attributes: rating, rating_class, weight, time")
print(f"\n📊 Gephi Visualization Tips:")
print(f"  1. Node size → pagerank (shows trust anchors)")
print(f"  2. Node color → community (shows clusters)")
print(f"  3. Edge color → rating_class (green=trust, red=distrust)")
print(f"  4. Layout → ForceAtlas2 (recommended for this network type)")

---

## 9. Executive Summary

### Key Findings

1. **Network Characteristics**:
   - The Bitcoin OTC network shows a healthy trust-dominated structure
   - J-curve rating distribution indicates typical reputation system behavior
   - Majority of ratings are positive (+10), with few distrust ratings

2. **Trust Anchors (PageRank)**:
   - Identified top 10 most trustworthy users based on network influence
   - PageRank successfully filters spam ratings by weighing quality over quantity
   - These users should be prioritized as trading partners

3. **Fraud Detection (Community Analysis)**:
   - Louvain algorithm detected community structure in the network
   - Small, isolated communities flagged as potential Sybil attack rings
   - Recommend manual review of suspicious communities

4. **Trust Path Analysis**:
   - Demonstrated ability to assess trading risk based on network connectivity
   - Users with no path to trust anchors represent higher risk
   - Can be used as pre-trade risk scoring mechanism

### Business Recommendations

1. **Implement Trust Tiers**:
   - Gold Tier: Top 10 PageRank users
   - Silver Tier: Users within 2 hops of Gold Tier
   - Bronze Tier: All other connected users
   - Unverified: Isolated users (require additional verification)

2. **Fraud Monitoring**:
   - Flag users in isolated communities for review
   - Monitor for sudden community formation (potential new attack)
   - Cross-reference with transaction patterns

3. **New User Onboarding**:
   - Encourage new users to get ratings from established trust anchors
   - Provide trust score visibility to help users make informed decisions
   - Implement graduated trading limits based on trust tier

### Technical Notes

- All algorithms handle negative edge weights appropriately
- Graph exported to Gephi for advanced visualization
- Network can be updated incrementally as new ratings arrive
- Algorithms are scalable to larger networks

---